# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.6638258569725202
 0.5609354856596882
 2.217216857740925 

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.44047    1.0       0.0
 0.244944  -0.181691  1.0
U factor:
3×3 Array{Float64,2}:
 0.931192  0.363442  0.922583
 0.0       0.211583  0.475625
 0.0       0.0       0.142702

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.44047    1.0       0.0
 0.244944  -0.181691  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.931192  0.363442  0.922583
 0.0       0.211583  0.475625
 0.0       0.0       0.142702

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0               
 0.9999999999999996
 1.0000000000000002

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0               
 0.9999999999999996
 1.0000000000000002

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.393338   0.903489  -0.170272
 -0.218733  -0.271843  -0.937154
 -0.892996  -0.331374   0.30455 
R factor:
3×3 Array{Float64,2}:
 -1.04277  -0.481807  -1.23253 
  0.0       0.201613   0.414422
  0.0       0.0       -0.133734

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.393338   0.903489  -0.170272
 -0.218733  -0.271843  -0.937154
 -0.892996  -0.331374   0.30455 

In [12]:
Aqr.R

3×3 Array{Float64,2}:
 -1.04277  -0.481807  -1.23253 
  0.0       0.201613   0.414422
  0.0       0.0       -0.133734

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -0.5683329813124303 
 -0.11188160922379536
  3.446864641257397  
eigenvectors:
3×3 Array{Float64,2}:
 -0.808793  -0.00192265  -0.58809 
  0.186685  -0.949112    -0.253642
  0.557676   0.314931    -0.767995

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Array{Float64,1}:
 -0.5683329813124303 
 -0.11188160922379536
  3.446864641257397  

In [15]:
AsymEig.vectors

3×3 Array{Float64,2}:
 -0.808793  -0.00192265  -0.58809 
  0.186685  -0.949112    -0.253642
  0.557676   0.314931    -0.767995

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
 1.0          0.0          4.44089e-16
 1.77636e-15  1.0          2.66454e-15
 8.88178e-16  5.55112e-16  1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.23109622793565632

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.206680 seconds (172.66 k allocations: 16.410 MiB, 24.55% gc time)


In [23]:
@time eigvals(Asym_noisy);

  0.535746 seconds (18 allocations: 7.921 MiB)


In [24]:
@time eigvals(Asym_explicit);

  0.071740 seconds (6.47 k allocations: 8.338 MiB, 7.42% gc time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.755191 seconds (560.30 k allocations: 211.339 MiB, 10.24% gc time)


6.691048792404253

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 9//10  1//5   4//5 
 1//5   7//10  7//10
 1//1   1//2   1//1 

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
 19//10
  8//5 
  5//2 

In [29]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
  1//1   0//1   0//1
  2//9   1//1   0//1
 10//9  25//59  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 9//10   1//5     4//5  
 0//1   59//90   47//90 
 0//1    0//1   -13//118

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [31]:
using LinearAlgebra

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback

Please click on `Validate` on the top, once you are done with the exercises.